# Подзапросы

###### Задание 1 

Напишите SQL-запрос, который выведет названия (***name***) сериалов, у которых первый эпизод первого сезона получил рейтинг 10. Назовите результирующее поле ***name***. Для выборки используйте таблицы ***titles***, ***ratings***, ***episodes*** и коррелированные подзапросы. Результат отсортируйте в лексикографическом порядке.

``` PLSQL
SELECT t.original_title as name
FROM   titles as t
WHERE  t.id in (SELECT e.parent_id
                FROM   episodes as e join ratings as r
                        ON e.title_id = r.title_id
                WHERE  e.season = 1
                   and e.episode = 1
                   and r.avg_rating = 10)
ORDER BY 1;
```
###### Задание 2 

Напишите SQL-запрос, который выведет оригинальное название (***original_title***), год начала (***year_of_start***) и количество эпизодов произведений, у которых больше 500 серий, отсортировав их по убыванию количества серий. Для этого используйте подзапрос к таблице ***episodes*** с группировкой по полю ***parent_id***. Выведите только строки, которые есть в таблице ***titles*** и в сгруппированной таблице.

Назовите поле с количеством эпизодов произведений ***ep_cnt***. Отсортируйте результат по этому полю в порядке убывания.

``` PLSQL
SELECT t.original_title,
       t.year_of_start,
       e_cnt.ep_cnt
FROM   titles as t join (SELECT parent_id,
                                count(*) as ep_cnt
                         FROM   episodes as e
                         GROUP BY parent_id having count(*) > 500) as e_cnt
        ON t.id = e_cnt.parent_id
ORDER BY 3 desc;
```

###### Задание 3

Напишите SQL-запрос, который выведет:

— количество строк в таблице ***titles*** (назовите поле ***title_cnt***);

— количество уникальных записей в поле type таблицы ***titles*** (назовите поле ***type_cnt***);

— общее количество голосов ***vote_cnt*** в таблице ***ratings*** (назовите поле ***sum_vote***).

``` PLSQL
SELECT count(*) as title_cnt,
       count (distinct type) as type_cnt,
       (SELECT sum(vote_cnt)
 FROM   ratings) as sum_vote
FROM   titles;
```


###### Задание 4

Напишите SQL-запрос, который с помощью коррелированного подзапроса выведет:

— ***id*** и ***original_title*** из таблицы ***titles***;

— ***avg_rating*** из таблицы ***ratings***, относящийся к строке из ***titles***.

Результат отсортируйте по возрастанию ***id***, ***original_title***, ***avg_rating***.

``` PLSQL
SELECT t.id,
       t.original_title,
       r.avg_rating
FROM   titles as t
    LEFT JOIN (SELECT *
               FROM   ratings) as r
        ON t.id = r.title_id
ORDER BY 1, 2, 3;
```

###### Задание 5

Напишите SQL-запрос, который с помощью подзапроса в блоке ***WHERE*** выведет строку таблицы ***titles***, у которой максимальный рейтинг (***avg_rating*** из таблицы ***ratings***) и максимальное количество голосов (***vote_cnt***) среди других произведений с тем же рейтингом.

``` PLSQL
SELECT *
FROM   titles
WHERE  id in (SELECT title_id
              FROM   ratings
              ORDER BY avg_rating desc, vote_cnt desc limit 1);
```

###### Задание 6

Напишите SQL-запрос, который выведет название (***original_title***) произведения с максимальной продолжительностью. Если таких произведений несколько, то выведите все.

``` PLSQL
SELECT original_title
FROM   titles
WHERE  run_time is not null
ORDER BY run_time desc limit 1;
```

###### Задание 7

Напишите SQL-запрос, который выведет названия (***original_title***) произведений, продолжительностью между ***floor*** и ***ceil*** средней продолжительности произведений в таблице ***titles***.

Результат отсортируйте по возрастанию ***original_title***.

``` PLSQL
SELECT original_title
FROM   titles
WHERE  run_time between (SELECT floor(avg(run_time))
                         FROM   titles) and (SELECT ceil(avg(run_time))
                    FROM   titles)
ORDER BY 1;
```

###### Задание 8

Напишите SQL-запрос, который выведет тип (***type***), оригинальное название (***original_title***), год выпуска (***year_of_start***) и жанр (***genres***) произведений для взрослых (***is_adult***), отсортировав их по убыванию рейтинга (***avg_rating*** в таблице ***ratings***). Для этого используйте коррелированный подзапрос в секции сортировки.

``` PLSQL
SELECT type,
       original_title,
       year_of_start,
       genres
FROM   titles as t
WHERE  is_adult = true
ORDER BY (SELECT avg_rating
          FROM   ratings as r
          WHERE  r.title_id = t.id) desc;
```

###### Задание 9

Напишите SQL-запрос, который выведет имена людей (поле ***name*** из таблицы ***persons***), которые судя по таблице ***principals*** участвовали в создании 10 самых высокооценённых произведений (максимум голосов среди тех, кто набрал максимальный рейтинг (***avg_rating***) из таблицы ***ratings***). Итоговый результат отсортируйте по ***name***.

``` PLSQL
SELECT name
FROM   persons
WHERE  id in(SELECT person_id
             FROM   principals
             WHERE  title_id in (SELECT title_id
                                 FROM   ratings
                                 ORDER BY avg_rating desc, vote_cnt desc limit 10))
ORDER BY 1;
```

###### Задание 10

Перепишите SQL-запрос из первого задания с использованием CTE. Запрос выводит названия (***original_name***) сериалов, у которых первый эпизод первого сезона получил рейтинг 10. Назовите результирующее поле ***name*** и отсортируйте результат по его возрастанию.

``` PLSQL
with res as (SELECT e.parent_id
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id
             WHERE  e.season = 1
                and e.episode = 1
                and r.avg_rating = 10)
SELECT t.original_title as name
FROM   titles as t join res
        ON t.id = res.parent_id
ORDER BY 1;
```
###### Задание 11

Перепишите SQL-запрос из второго задания с использованием CTE. 

Запрос выводит оригинальное название (***original_title***), год начала (***year_of_start***) и количество эпизодов произведений (***ep_cnt***), у которых больше 500 серий, отсортировав их по убыванию количества серий. Выводятся только те строки, которые есть в таблице ***titles*** и в сгруппированной таблице.

``` PLSQL
with e as (SELECT parent_id,
                  count(1) as ep_cnt
           FROM   episodes as e
           GROUP BY parent_id having count(1) > 500)
SELECT t.original_title,
       t.year_of_start,
       e.ep_cnt
FROM   titles as t join e
        ON t.id = e.parent_id
ORDER BY ep_cnt desc;
```
